In [1]:
!pip install -q emoji
!pip install -q datasets
!pip install -q evaluate
!pip install -q rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platf

In [10]:
import os
import re
import json
import emoji
import torch
import evaluate
import numpy as np
from tqdm.notebook import tqdm
from datasets import load_dataset, Dataset
from transformers import (
    BartTokenizerFast,
    BartForConditionalGeneration,
    BartConfig,
    EarlyStoppingCallback,
    TrainerCallback,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the codeparrot/apps from HuggingFace

In [3]:
dataset = load_dataset('codeparrot/apps')
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.63k [00:00<?, ?B/s]

apps.py:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

The repository for codeparrot/apps contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/apps.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.jsonl:   0%|          | 0.00/107M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['problem_id', 'question', 'solutions', 'input_output', 'difficulty', 'url', 'starter_code'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['problem_id', 'question', 'solutions', 'input_output', 'difficulty', 'url', 'starter_code'],
        num_rows: 5000
    })
})

In [4]:
sample_idx = 0  # Change this index to view different samples
print(f"Problem ID: {dataset['train'][sample_idx]['problem_id']}")
print(f"Difficulty: {dataset['train'][sample_idx]['difficulty']}")
print(f"Problem Statement:\n{dataset['train'][sample_idx]['question']}")
print(f"Solution:\n{dataset['train'][sample_idx]['solutions']}")

Problem ID: 0
Difficulty: interview
Problem Statement:
Polycarp has $n$ different binary words. A word called binary if it contains only characters '0' and '1'. For example, these words are binary: "0001", "11", "0" and "0011100".

Polycarp wants to offer his set of $n$ binary words to play a game "words". In this game, players name words and each next word (starting from the second) must start with the last character of the previous word. The first word can be any. For example, these sequence of words can be named during the game: "0101", "1", "10", "00", "00001".

Word reversal is the operation of reversing the order of the characters. For example, the word "0111" after the reversal becomes "1110", the word "11010" after the reversal becomes "01011".

Probably, Polycarp has such a set of words that there is no way to put them in the order correspondent to the game rules. In this situation, he wants to reverse some words from his set so that:  the final set of $n$ words still contains

In [5]:
# Check the number of test cases per problem
test_case_counts = [len(item['input_output']) for item in tqdm(dataset['train']) if 'input_output' in item]
print(f"Min number of test cases: {min(test_case_counts) if test_case_counts else 'N/A'}")
print(f"Max number of test cases: {max(test_case_counts) if test_case_counts else 'N/A'}")
print(f"Average number of test cases: {np.mean(test_case_counts) if test_case_counts else 'N/A'}")

  0%|          | 0/5000 [00:00<?, ?it/s]

Min number of test cases: 0
Max number of test cases: 23613166
Average number of test cases: 5749.3512


In [6]:
no_test_cases = sum(1 for item in dataset['train'] if 'input_output' not in item or len(item['input_output']) == 0)
no_solutions = sum(1 for item in dataset['test'] if 'solutions' not in item or len(item['solutions']) == 0)
print(f'Problems without test cases in train split: {no_test_cases}')
print(f'Problems without solutions in test split: {no_solutions}')

Problems without test cases in train split: 195
Problems without solutions in test split: 1235


# Split and Clean the Data

In [7]:
def clean_question(question):
    question = re.sub(r'<[^>]*>', '', question) # Remove HTML
    question = emoji.replace_emoji(question, '') # Remove Emoji
    question = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)', '', question) # Remove URL
    question = re.sub(r'[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+', '', question) # Remove phone email
    question = re.sub(r'^[\+]?[(]?[0-9]{3}[)]?[-\s\.]?[0-9]{3}[-\s\.]?[0-9]{4,6}$', '', question) # Remove phone number
    question = re.sub(r'#\w+', '', question) # Remove hashtags
    question = re.sub(r"[^\sa-zA-Z0-9]", ' ', question) # Remove unnecessary characters
    question = re.sub(r'\n\n+s*\n', '\n', question) # Remove extra newline
    return re.sub(r'\s+', ' ', question).strip() # Remove extra whitespace

In [8]:
def clean_solution(solution):
    solution = re.sub(r'#\s*Time:.*|#\s*Space:.*|#\s*@author:.*|#\s*@date:.*', '', solution) # Remove comments that don't add value
    if 'def ' in solution or 'class ' in solution: # Skip imports, handle only function/class definitions
        # Try to find the first function or class definition
        first_def = solution.find('def ')
        first_class = solution.find('class ')

        # Find the earliest occurrence of either def or class
        start_idx = min(x for x in [first_def, first_class] if x >= 0) if first_def >= 0 or first_class >= 0 else 0
        solution = solution[start_idx:]

    # Remove trailing whitespace and ensure consistent newlines
    solution = '\n'.join(line.rstrip() for line in solution.strip().splitlines())
    return solution

In [11]:
# Split the train dataset into train and validation at the problem level to avoid leakage
# and create (question, solution) pairs, one per solution, for training and validation
train_val_split = dataset['train'].train_test_split(test_size=0.1, seed=42)
num_of_solutions = 1 # Number of solutions to take per question

train_data = [{'question': clean_question(sample['question']), 'solution': clean_solution(solution)}
    for sample in tqdm(train_val_split['train'])
    for solution in json.loads(sample['solutions'])[-num_of_solutions:]
]
val_data = [
    {'question': clean_question(sample['question']), 'solution': clean_solution(solution)}
    for sample in tqdm(train_val_split['test'])
    for solution in json.loads(sample['solutions'])[-num_of_solutions:]
]
test_data = [
    {'question': clean_question(sample['question']), 'solution': clean_solution(solution)}
    for sample in tqdm(dataset['test']) if sample['solutions']
    for solution in json.loads(sample['solutions'])[-num_of_solutions:]
]

  0%|          | 0/4500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [12]:
processed_data_path = 'processed_data'
os.makedirs(processed_data_path, exist_ok=True)

for split_name, split_data in zip(['train', 'val', 'test'], [train_data, val_data, test_data]):
    output_file = os.path.join(processed_data_path, f'{split_name}.json')
    with open(output_file, 'w') as f: # Save processed data splits to files
        json.dump(split_data, f, indent=2)
print(f'Extracted {len(train_data)} train, {len(val_data)} validation, and {len(test_data)} test examples')

Extracted 4500 train, 500 validation, and 3765 test examples


# Retrain the Tokenizer

In [13]:
# Extract questions and solutions from the train split to create a domain-specific corpus
questions = [sample['question'] for sample in dataset['train']]
solutions = [sol for sample in dataset['train'] for sol in json.loads(sample['solutions'])]

In [14]:
# Initialize a base tokenizer and train a new 1 on our corpus
base_tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
new_tokenizer = base_tokenizer.train_new_from_iterator(
    questions + solutions, # Combine natural language and code
    vocab_size = 50265  # Match model's original vocab size for compatibility
)
new_tokenizer.pad_token = new_tokenizer.eos_token
new_tokenizer.save_pretrained('apps_tokenizer')  # Save the retrained tokenizer

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

('apps_tokenizer/tokenizer_config.json',
 'apps_tokenizer/special_tokens_map.json',
 'apps_tokenizer/vocab.json',
 'apps_tokenizer/merges.txt',
 'apps_tokenizer/added_tokens.json',
 'apps_tokenizer/tokenizer.json')

In [15]:
new_tokenizer = BartTokenizerFast.from_pretrained('apps_tokenizer')
test_input = 'def solve(nums):\n    return sum(nums)'
encoded = new_tokenizer.encode(test_input)
decoded = new_tokenizer.decode(encoded)
print(f'Testing tokenizer:\n'
      f'Original: {test_input}\n'
      f'Encoded: {encoded}\n'
      f'Decoded: {decoded}\n'
      f'Vocabulary size: {new_tokenizer.vocab_size}')

Testing tokenizer:
Original: def solve(nums):
    return sum(nums)
Encoded: [0, 314, 1178, 12, 624, 286, 276, 299, 504, 12, 624, 13, 2]
Decoded: <s>def solve(nums):
    return sum(nums)</s>
Vocabulary size: 50265


# Tokenization for Sequence-to-Sequence Task

In [16]:
def tokenize_function(example): # tokenization function
    inputs = new_tokenizer(example['question'], truncation=True, max_length=512) # Tokenize inputs (questions)
    labels = new_tokenizer(example['solution'], truncation=True, max_length=512) # Tokenize targets (solutions)

    labels_with_ignore = [] # Replace padding token id with -100 so it's ignored in the loss
    for label in labels['input_ids']:
        labels_with_ignore.append([-100 if token == new_tokenizer.pad_token_id else token for token in label])
    inputs['labels'] = labels_with_ignore
    return inputs

In [17]:
train_dataset_processed = Dataset.from_list(train_data)
val_dataset_processed = Dataset.from_list(val_data)

tokenized_train_dataset = train_dataset_processed.map(tokenize_function, batched=True, remove_columns=train_dataset_processed.column_names)
tokenized_val_dataset = val_dataset_processed.map(tokenize_function, batched=True, remove_columns=val_dataset_processed.column_names)
tokenized_val_dataset # Dynamic padding will be handled by DataCollator

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})

In [ ]:
# labels = np.array(tokenized_val_dataset[3]['labels'])
# labels = np.where(labels != -100, labels, new_tokenizer.pad_token_id)
# print(new_tokenizer.decode(labels))

# Metrics

In [18]:
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [19]:
def preprocess_logits_for_metrics(logits, labels):
    '''
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/15
    '''
    if isinstance(logits, tuple): pred_ids = logits[0]
    else: pred_ids = logits
    if pred_ids.ndim == 3: pred_ids = torch.argmax(pred_ids, dim=-1)
    return pred_ids, labels

In [20]:
def compute_metrics(eval_preds):
    preds = eval_preds.predictions[0]
    labels = eval_preds.label_ids
    preds = np.where(preds != -100, preds, new_tokenizer.pad_token_id) # Replace -100 with pad token id
    labels = np.where(labels != -100, labels, new_tokenizer.pad_token_id) # Replace -100 with pad token id

    # Decode predictions and labels
    decoded_preds = new_tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = new_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU, ROUGE, and exact match score
    bleu_results = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_results = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    # exact_match = sum(pred == label for pred, label in zip(decoded_preds, decoded_labels)) / len(decoded_preds)

    return {
        'bleu': bleu_results['bleu'],
        'rouge1': rouge_results['rouge1'],
        'rouge2': rouge_results['rouge2'],
        'rougeL': rouge_results['rougeL'],
        'meteor': meteor_results['meteor'],
        # 'exact_match': exact_match,
    }

In [21]:
class PerplexityCallback(TrainerCallback): # Define callback to compute perplexity from eval_loss
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if 'eval_loss' in metrics:
            perplexity = torch.exp(torch.tensor(metrics['eval_loss']))
            metrics['perplexity'] = perplexity.item()

# Training Setup

In [22]:
# config = BartConfig.from_pretrained(
#     'facebook/bart-base',
#     vocab_size=new_tokenizer.vocab_size,
#     # max_position_embeddings=1024,
#     # encoder_layers=6,
#     # decoder_layers=6,
#     # encoder_attention_heads=12,
#     # decoder_attention_heads=12,
#     # encoder_ffn_dim=3072,
#     # decoder_ffn_dim=3072,
#     # d_model=768
# )
# model = BartForConditionalGeneration(config) # Initialize a new model with this configuration
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model.resize_token_embeddings(len(new_tokenizer))
total_params = sum(p.numel() for p in model.parameters())
print(f'Model initialized with {total_params / 1e6:.2f}M parameters')

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Model initialized with 139.42M parameters


In [23]:
training_args = Seq2SeqTrainingArguments( # Define training arguments for fine-tuning
    output_dir='./results',               # Directory for checkpoints and logs
    num_train_epochs=20,                  #
    per_device_train_batch_size=32,       # Batch size per GPU
    per_device_eval_batch_size=32,        # Evaluation batch size
    learning_rate=2e-4,
    # lr_scheduler_type='cosine',
    weight_decay=0.01,                    # Regularization
    logging_strategy='epoch',             #
    eval_strategy='epoch',                # Evaluate after each epoch
    save_strategy='epoch',                # Save after each epoch
    # predict_with_generate=True,          # Whether to use generate to calculate generative metrics (ROUGE, BLEU)
    # generation_max_length=512,           #
    load_best_model_at_end=True,          # Load the best model based on validation loss
    metric_for_best_model='eval_loss',    # Use validation loss for early stopping
    greater_is_better=False,              # Lower loss is better
    fp16=torch.cuda.is_available(),       # Enable mixed-precision training if a CUDA GPU is available (faster, less memory)
)

# Fine-tune the Model

In [24]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=new_tokenizer, model=model), # Set up data collator for dynamic padding
    processing_class=new_tokenizer,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    callbacks=[PerplexityCallback, EarlyStoppingCallback(early_stopping_patience=3)]
)
trainer.train()  # Perform the fine-tuning
trainer.save_model('trained_model')  # Save the fine-tuned model

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 18520339 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor,
1,4.444600,3.178368,0.096479,0.547477,0.223142,0.423305,0.289886,24.007532
2,2.941900,2.795707,0.208532,0.355200,0.158744,0.269087,0.385159,16.374205
3,2.522600,2.578743,0.160712,0.490950,0.227605,0.387721,0.369820,13.180566
4,2.258700,2.485482,0.239366,0.424287,0.199883,0.331328,0.420790,12.006911
5,2.048200,2.467089,0.245605,0.609942,0.296851,0.488508,0.432780,11.788078
6,1.869100,2.429100,0.378947,0.594005,0.287043,0.480538,0.479214,11.348667
7,1.711400,2.440459,0.359005,0.610585,0.299181,0.498203,0.475599,11.478309
8,1.564800,2.452678,0.360102,0.561823,0.274919,0.458774,0.474330,11.619427
9,1.423300,2.478424,0.323725,0.445377,0.217947,0.361017,0.464837,11.922463


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


# Evaluation on Test Set

In [25]:
test_dataset_processed = Dataset.from_list(test_data)
tokenized_test_dataset = test_dataset_processed.map(tokenize_function, batched=True, remove_columns=test_dataset_processed.column_names)
trainer.evaluate(tokenized_test_dataset, metric_key_prefix='test')

Map:   0%|          | 0/3765 [00:00<?, ? examples/s]

early stopping required metric_for_best_model, but did not find eval_loss so early stopping is disabled


{'test_loss': 2.139223575592041,
 'test_bleu': 0.4127579628198951,
 'test_rouge1': 0.6549011446562243,
 'test_rouge2': 0.33177189468932516,
 'test_rougeL': 0.5192459999330195,
 'test_meteor': 0.5150344160175295,
 'test_runtime': 65.3705,
 'test_samples_per_second': 57.595,
 'test_steps_per_second': 1.805,
 'epoch': 9.0}

# Inference

In [26]:
def generate_code(model, tokenizer, problem_text, device, max_length=512, num_beams=5, top_p=0.95, temperature=0.7):
    # Generate code solution for a given problem using beam search
    inputs = tokenizer(problem_text, return_tensors='pt', truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    output = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=num_beams,
        top_p=top_p,
        temperature=temperature,
        no_repeat_ngram_size=2,
        do_sample=True,
        early_stopping=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [27]:
question, solution = test_data[0]['question'], test_data[0]['solution']
pred_solution = generate_code(model, new_tokenizer, question, device)
print(question)
print(f'\n===== Grounth Truth: =====\n{solution}')
print(f'\n===== Prediction: =====\n{pred_solution}')

An accordion is a string yes in the real world accordions are musical instruments but let s forget about it for a while which can be represented as a concatenation of an opening bracket ASCII code 091 a colon ASCII code 058 some possibly zero vertical line characters ASCII code 124 another colon and a closing bracket ASCII code 093 The length of the accordion is the number of characters in it For example and are accordions having length 4 6 and 7 are not accordions You are given a string s You want to transform it into an accordion by removing some possibly zero characters from it Note that you may not insert new characters or reorder existing ones Is it possible to obtain an accordion by removing characters from s and if so what is the maximum possible length of the result Input The only line contains one string s 1 le s le 500000 It consists of lowercase Latin letters and characters and Output If it is not possible to obtain an accordion by removing some characters from s print 1 Oth